Nosso projeto consiste em avaliar o processo de visto americano para trabalho, o H1B, e se dependendo de certos fatores uma pessa receberia o visto ou não. Para tirar este visto é necessário passar informações como, o nome do lugar de trabalho, com o que a pessoa trabalhará, se é um trabalho de tempo integral, o salário, e a cidade onde será o trabalho (traduzido para latitude e longitude), todos dados contidos no nosso dataset. Porém havia um problema nosso dataset, obtido no Kaggle havia 3 milhões de linhas, um arquivo muito grande, fazendo com que fique difícil e demorado para se trabalhar. Por esse motivo cortamos o dataset, no qual o processo esta em um outro notebook chamado Shuffle dataset.

In [8]:
import pandas as pd
import numpy as np
import sklearn

In [9]:
df = pd.read_csv('Dataframe.csv')
df.head()

,Unnamed: 0,Unnamed: 0.1,CASE_STATUS,EMPLOYER_NAME,SOC_NAME,JOB_TITLE,FULL_TIME_POSITION,PREVAILING_WAGE,YEAR,WORKSITE,lon,lat
0,0,353010,1,HYADES TECHNOLOGY INC,"SOFTWARE DEVELOPERS, APPLICATIONS",SOFTWARE DEVELOPER,Y,76107.0,2016.0,"REDWOOD CITY, CALIFORNIA",-122.236355,37.485215
1,1,1679744,0,"AXIOM SOFTWARE LABOARATORIES, INC","Financial Specialists, All Other",REGULATORY REPORTING SPECIALIST,Y,81640.0,2014.0,"NEW YORK, NEW YORK",-74.005941,40.712784
2,2,2065634,0,OLLER EXECUTIVE GROUP INC.,Accountants and Auditors,ACCOUNTANT,Y,55000.0,2013.0,"JAMAICA PLAIN, MASSACHUSETTS",-71.115143,42.309736
3,3,490742,1,SUNPOWER CORPORATION,"COMPUTER OCCUPATIONS, ALL OTHER",PROJECT MANAGER,N,52790.0,2016.0,"AUSTIN, TEXAS",-97.743061,30.267153
4,4,1160570,0,BELUGA NORTH AMERICA INC,PUBLIC RELATIONS AND FUNDRAISING MANAGERS,PUBLIC RELATIONS MANAGER,Y,92456.0,2015.0,"NORTH MIAMI BEACH, FLORIDA",-80.162546,25.933149


Na célula abaixo, o tratamento dos dados é feito. Em primeiro lugar as colunas Unnamed: 0 e Unnamed: 0.1, são retiradas, essas correspondem ao index atual e ao index antes do embaralhamento feito no outro notebook. O local de trabalho, WORKSITE, também é retirado uma vez que esse já está no dataset em forma de Latitude e Longitude, variáves quantitavas. Essas então são transformadas para valores de 0 a 1 para facilitar a forma de se trabalhar, bem como a variável de trabalho em tempo intergral, FULL_TIME_POSITION, também é transformada, de N ou Y, para 0 ou 1. JOB_TITLE é retirado pois a variável SOC_NAME, do Inglês Standard Occupational Classification (SOC), já corresponde ao trabalho ocupado porém em uma forma de classificação padrão e essa então é transormada para uma váriavel quantitativa através do metodo de One-hot Encoding. Por fim o ano é retirado uma vez que queremos prever o visto de anos futuros então não se pode levar em conta os anos passados.  

In [10]:
def dummify(data, column_name):
    df = data.copy()
    df2 = pd.concat([df.drop(column_name, axis=1), pd.get_dummies(data[column_name], prefix=column_name)], axis=1)
    return df2

def mapp (value, start1, stop1, start2, stop2):
    return start2 + (stop2 - start2) * ((value - start1) / (stop1 - start1))


df = df.drop(["WORKSITE","YEAR","JOB_TITLE","Unnamed: 0","Unnamed: 0.1"], axis=1)
df["SOC_NAME"] = df["SOC_NAME"].apply(lambda x: x.upper())

df = df.drop(["EMPLOYER_NAME"], axis=1)

df["FULL_TIME_POSITION"] = df["FULL_TIME_POSITION"].apply(lambda x: 0 if x=="N" else 1)



latmin = df["lat"].min()
latmax = df["lat"].max()
lonmin = df["lon"].min()
lonmax = df["lon"].max()

df["lat"] = df["lat"].apply(lambda x: mapp(x, latmin, latmax, 0, 1))
df["lon"] = df["lon"].apply(lambda x: mapp(x, lonmin, lonmax, 0, 1))

df = dummify(df, "SOC_NAME")

In [11]:
df.head()

,CASE_STATUS,FULL_TIME_POSITION,PREVAILING_WAGE,lon,lat,SOC_NAME_ACCOUNTANTS,SOC_NAME_ACCOUNTANTS AND AUDITORS,SOC_NAME_ACTUARIES,SOC_NAME_ADMINISTRATIVE SERVICES MANAGERS,SOC_NAME_ADULT BASIC AND SECONDARY EDUCATION AND LITERACY,...,SOC_NAME_VETERINARIANS,SOC_NAME_VETERINARY TECHNOLOGISTS AND TECHNICIANS,SOC_NAME_VOCATIONAL EDUCATION TEACHERS POSTSECONDARY,"SOC_NAME_VOCATIONAL EDUCATION TEACHERS, POSTSECONDARY",SOC_NAME_WEB ADMINISTRATORS,SOC_NAME_WEB DEVELOPERS,"SOC_NAME_WELDERS, CUTTERS, SOLDERERS, AND BRAZERS","SOC_NAME_WHOLESALE AND RETAIL BUYERS, EXCEPT FARM PRODUCTS",SOC_NAME_WRITERS AND AUTHORS,SOC_NAME_ZOOLOGISTS AND WILDLIFE BIOLOGISTS
0,1,1,76107.0,0.117337,0.467855,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,1,81640.0,0.276204,0.530647,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,1,55000.0,0.285727,0.561716,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,1,0,52790.0,0.198016,0.327427,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,1,92456.0,0.255925,0.243109,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


O X e y do modelo, baseando-se no CASE_STATUS(se foi aprovado ou não) são separados em base para treinamentos e base para teste, sendo a divisão feita de 80 e 20 por cento. Em seguida é aplicado o modelo, neste caso de regressão logística, uma vez que esse método consegue fazer a predição de para valores a partir das nossas variáves que foram transformadas em quantitativas.

In [12]:
col = []

for i in df.columns:
    if i.startswith("SOC_NAME_"):
        col.append(i)

X = df.drop(["CASE_STATUS"], axis=1).values
y = df["CASE_STATUS"].values

index = round(df.index.size*0.8)

X_train, X_test = X[:index], X[index:]
y_train, y_test = y[:index], y[index:]

In [13]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score


model = LogisticRegression(solver="lbfgs")
model.fit(X_train, y_train)
y_pred = model.predict(X_test)
print("Acurácia: {}".format(100*accuracy_score(y_test, y_pred)))

Acurácia: 50.0


In [14]:
df.head()

,CASE_STATUS,FULL_TIME_POSITION,PREVAILING_WAGE,lon,lat,SOC_NAME_ACCOUNTANTS,SOC_NAME_ACCOUNTANTS AND AUDITORS,SOC_NAME_ACTUARIES,SOC_NAME_ADMINISTRATIVE SERVICES MANAGERS,SOC_NAME_ADULT BASIC AND SECONDARY EDUCATION AND LITERACY,...,SOC_NAME_VETERINARIANS,SOC_NAME_VETERINARY TECHNOLOGISTS AND TECHNICIANS,SOC_NAME_VOCATIONAL EDUCATION TEACHERS POSTSECONDARY,"SOC_NAME_VOCATIONAL EDUCATION TEACHERS, POSTSECONDARY",SOC_NAME_WEB ADMINISTRATORS,SOC_NAME_WEB DEVELOPERS,"SOC_NAME_WELDERS, CUTTERS, SOLDERERS, AND BRAZERS","SOC_NAME_WHOLESALE AND RETAIL BUYERS, EXCEPT FARM PRODUCTS",SOC_NAME_WRITERS AND AUTHORS,SOC_NAME_ZOOLOGISTS AND WILDLIFE BIOLOGISTS
0,1,1,76107.0,0.117337,0.467855,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,1,81640.0,0.276204,0.530647,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,1,55000.0,0.285727,0.561716,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,1,0,52790.0,0.198016,0.327427,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,1,92456.0,0.255925,0.243109,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
